# Dense Embeddings with Propietary Models

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from dotenv import load_dotenv
import pandas as pd
import tiktoken
from openai import OpenAI
from qdrant_client import models, QdrantClient


from src import load_data, embeddings, utils


c:\Users\manuelalberto.romero\Documents\repos\dslabs\mbit\peia-oct25\dslab-rag-retrieval\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\manuelalberto.romero\Documents\repos\dslabs\mbit\peia-oct25\dslab-rag-retrieval\src\load_data.py:18: SyntaxWarning: invalid escape sequence '\d'
  suffix="\d+"  # forma del sufijo: numero


# Params

In [20]:
INDEX = "ragquas-openai-01"
EMB_MODEL = "text-embedding-3-small"
EMB_DIM = 1536
RETRIEVE_K=5

LEN_MAX_TOKENS = 2048
SMPL_SIZE = 25  # int or None

# Environment variables

In [21]:
load_dotenv()

OPENAI_API_KEY = os.environ['OPENAI_API_KEY']


# Clients

In [22]:
client_openai = OpenAI(
    api_key=OPENAI_API_KEY
)

client_qdrant = QdrantClient(":memory:") 


In [6]:
encoding = tiktoken.get_encoding("cl100k_base")
encoding = tiktoken.encoding_for_model(EMB_MODEL)

# Load and Prepare Data

In [23]:
df_ragquas = load_data.ragquas()
df_ragquas.head(2)

,topic,answer,question,variant,context_1,context_2,context_3,context_4,context_5,link_1,link_2,link_3,link_4,link_5,text_1,text_2,text_3,text_4,text_5
0,reclamaciones,La opción más fácil y eficaz para reclamar una...,¿Cuál es la forma más fácil de reclamar cuando...,question_1,#1. Airhelp\n\nLa empresa Airhelp fue fundada ...,En AirHelp hemos ayudado a más de 16 millones ...,"MYFLYRIGHT, expertos en derechos de los viajer...",,,https://www.businessinsider.es/mejores-paginas...,https://www.airhelp.com/es/retrasos-de-vuelos/,https://myflyright.com/es/servicios/vuelo-retr...,,,5 páginas donde poder reclamar tus vuelos por ...,Conoce las compensaciones cuando se retrasa tu...,Indemnización retraso vuelo\n\nNavegación por ...,,
1,reclamaciones,La opción más fácil y eficaz para reclamar una...,hola mi vuelo a Alemania se ha retrasado mucho...,question_2,#1. Airhelp\n\nLa empresa Airhelp fue fundada ...,En AirHelp hemos ayudado a más de 16 millones ...,"MYFLYRIGHT, expertos en derechos de los viajer...",,,https://www.businessinsider.es/mejores-paginas...,https://www.airhelp.com/es/retrasos-de-vuelos/,https://myflyright.com/es/servicios/vuelo-retr...,,,5 páginas donde poder reclamar tus vuelos por ...,Conoce las compensaciones cuando se retrasa tu...,Indemnización retraso vuelo\n\nNavegación por ...,,


In [24]:
df_docs = load_data.prepare_ragquas(df_ragquas)
df_docs.info()

Raw shape: (250, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183 entries, 0 to 182
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   index     183 non-null    int64 
 1   topic     183 non-null    object
 2   variant   183 non-null    object
 3   question  183 non-null    object
 4   answer    183 non-null    object
 5   j         183 non-null    int64 
 6   context_  183 non-null    object
 7   text_     183 non-null    object
 8   link_     183 non-null    object
 9   uuid      183 non-null    object
dtypes: int64(2), object(8)
memory usage: 14.4+ KB


In [25]:
if SMPL_SIZE:
    df_docs_smpl = df_docs.iloc[:SMPL_SIZE]
else:
    df_docs_smpl = df_docs
    
lst_texts = df_docs_smpl["text_"].tolist()

encoding = tiktoken.encoding_for_model(EMB_MODEL)
len(encoding.encode(lst_texts[0]))


886

# Encode

In [10]:
# corpus_lens = utils.corpus_stats(encoding, df_docs_smpl['text_'])
# corpus_stats = corpus_lens.describe()
# corpus_stats

In [ ]:
lst_texts_tkns_len = []
lst_texts_tkns = []

for text in lst_texts:
    text_tkns = encoding.encode(text)  # tokenized [114, 29230, 123,...]
    ntokens = len(text_tkns)  # n tokens
    lst_texts_tkns.append(text_tkns)
    lst_texts_tkns_len.append(ntokens)  # list of longitudes en tokens


pd.Series(lst_texts_tkns_len).describe()


count      25.000000
mean     3115.960000
std      2496.030423
min       542.000000
25%      1472.000000
50%      2339.000000
75%      3780.000000
max      8768.000000
dtype: float64

In [12]:
lst_texts_tkns_trunc = []

for text in lst_texts_tkns:
    lst_texts_tkns_trunc.append(text[:LEN_MAX_TOKENS])

In [13]:
response = client_openai.embeddings.create(
    input=lst_texts_tkns_trunc,
    model=EMB_MODEL
)



In [ ]:
# type(response)

openai.types.create_embedding_response.CreateEmbeddingResponse

In [ ]:
# len(response.data[0].embedding)

1536

In [14]:
lst_embeddings = []

for vec in response.data:
    lst_embeddings.append(vec.embedding)


print(f"{len(lst_embeddings)}")
print(f"{len(lst_embeddings[0])}")  # embeddings dimension

25
1536


# Index

In [15]:
lst_pts = embeddings.create_index_points(
    lst_embeddings,
    df_docs_smpl,
)  # [{"id": ..., "payload": ..., "vector": }]
print(f"{len(lst_pts)=}")

len(lst_pts)=25


In [16]:
lst_qdrant_pts = embeddings.convert_to_qdrant_points(lst_pts)
print(f"{len(lst_qdrant_pts)=}")

len(lst_qdrant_pts)=25


In [34]:
lst_qdrant_pts[0].id

'7ea8fd27-86ca-4abe-850e-038565d2e9b5'

In [35]:
lst_qdrant_pts[0].vector[:5]

[-0.016400696709752083,
 0.03485557809472084,
 0.020323315635323524,
 0.037499796599149704,
 -0.029457880184054375]

In [36]:
lst_qdrant_pts[0].payload

{'text': '5 páginas donde poder reclamar tus vuelos por retraso o cancelación de manera gratuita\nIrina Pérez\n\n    En muchas ocasiones no se realiza la reclamación de un vuelo por retraso o cancelación, porque se desconocen los derechos como pasajeros y la manera de realizar el trámite. \n    Por ello, aquí van 5 reclamadores online que te ayudarán a conseguir una indemnización por tu vuelo, sin que tengas que desplazarte ni hacer papeleos.\n    Descubre más historias en Business Insider España.\n\nVolar es siempre sinónimo de nervios, normalmente ir al aeropuerto con tiempo, las colas en facturación, o cualquier imprevisto, hacen que a veces viajar sea una odisea. En el peor de los casos tu vuelo puede sufrir retrasos e incluso cancelarse. Pero, ¿sabías que tienes derecho a una compensación económica cuando esto ocurre?\n\nEn muchas ocasiones reclamar, es algo que se deja pasar porque es tedioso, y requiere hacer papeleo, desplazamientos y temas burocráticos que ocupan mucho tiempo.

In [17]:
client_qdrant.create_collection(
    collection_name=INDEX,
    vectors_config=models.VectorParams(
        size=EMB_DIM,
        distance=models.Distance.COSINE,  # with openai, vectors are normalized, could use dot product
    ),
)

client_qdrant.upsert(
    collection_name=INDEX,
    points=lst_qdrant_pts
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

# Query

In [38]:
df_docs_smpl.iloc[0]['question']

'¿Cuál es la forma más fácil de reclamar cuando un vuelo que sale de España se ha retrasado?'

In [18]:
_q_text = df_docs_smpl.iloc[0]['question']

response = client_openai.embeddings.create(
    input=_q_text,
    model=EMB_MODEL
)

_q_vec = response.data[0].embedding

print(f"Question: {_q_text}")
embeddings.check_query_qdrant(
        client_qdrant,
        INDEX,
        _q_vec,
        retrieve_k=3
)



Question: ¿Cuál es la forma más fácil de reclamar cuando un vuelo que sale de España se ha retrasado?
point.id='7ea8fd27-86ca-4abe-850e-038565d2e9b5'
point.score=0.6701799936516917
Doc: 5 páginas donde poder reclamar tus vuelos por retraso o cancelación de manera gratuita
Irina Pérez

    En muchas ocasiones no se realiza la reclamación de un vuelo por retraso o cancelación, porque se desconocen los derechos como pasajeros y la manera de realizar el trámite. 
    Por ello, aquí van 5 reclamadores online que te ayudarán a conseguir una indemnización por tu vuelo, sin que tengas que desplazarte ni hacer papeleos.
    Descubre más historias en Business Insider España.

Volar es si...
idx: 0, j: 1
------------------------------
point.id='018c7fad-e0ee-4b90-aa67-1af4c604d33b'
point.score=0.6507350878107978
Doc: Conoce las compensaciones cuando se retrasa tu vuelo

Llegas al aeropuerto y tu vuelo va con retraso, ¿conoces cuáles son tus derechos y si puedes pedir reclamar una compensación? De